In [2]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt

In [3]:
test_matrix = np.array([[1, 2], [3, 4]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [4]:
def kron(A, B):
    return la.kron(A, B)

In [5]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(B, np.eye(A.shape[0]))
    
    return L+R

In [6]:
print(kron(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

[[ 1  2  2  4]
 [ 3  4  6  8]
 [ 3  6  4  8]
 [ 9 12 12 16]]


In [7]:
n = 3
p_num = 100
eps_G = 10 ** (-6)
eps_Phi = 10 ** (-6)

In [8]:
# Входной BMAP
matrD_0 = np.matrix([[-86., 0.01], [0.02, -2.76]])
matrD = np.matrix([[85, 0.99], [0.2, 2.54]])
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

q = 0.8
for k in range(1, n+1):
    matrD_k.append(matrD * (q ** (k-1)) * (1 - q) / (1 - q ** 3))

print(matrD_0)

[[-0.2  0.6]
 [ 0.1 -0.4]]


In [9]:
# Поток поломок MMAP
matrH0 = np.matrix([[-8.110725, 0], [0, -0.26325]])
matrH1 = np.matrix([[8.0568, 0.053925], [0.146625, 0.116625]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1

In [10]:
# Поток обслуживания PH1
beta1 = np.matrix([1, 0])
matrS1 = np.matrix([[-20, 20], [0, -20]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.matrix([[1], [1]])
matrS1_0 = - matrS1 * M1_e

In [11]:
# Поток обслуживания PH2
beta2 = np.matrix([1, 0])
matrS2 = np.matrix([[-2, 2], [0, -2]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.matrix([[1], [1]])
matrS2_0 = - matrS2 * M2_e

In [12]:
matrS_w = kron(matrS1_0 * beta1, M2_e * beta2) + kron(M1_e * beta1, matrS2_0 * beta2)

In [13]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.matrix([0.05, 0.95])
matrA1 = np.matrix([[-1.86075, 0.], [0., -146.9994]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.matrix([[1], [1]])
matrA1_0 = - matrA1.dot(L1_e)

In [14]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.matrix([0.05, 0.95])
matrA2 = np.matrix([[-5.58225, 0.], [0., -440.9982]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.matrix([[1], [1]])
matrA2_0 = - matrA2.dot(L2_e)

In [15]:
# Поток ремонта PH
tau = np.matrix([0.98, 0.02])
matrT = np.matrix([[-100., 0.], [0., -0.0002]])
T_e = np.matrix([[1], [1]])
matrT0 = - matrT.dot(T_e)

R = matrT.shape[0]
R_ = R + 1

In [171]:
a = W_ * V_

In [104]:
# Q~0
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block00.shape[1] + block11.shape[1] + block22.shape[1] + block33.shape[1])

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
pd.DataFrame(matrQw_0)

36


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-8.310725,-0.000000,0.60000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000000,-8.310725,0.00000,0.60000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.100000,0.000000,-0.66325,-0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
3,0.000000,0.100000,-0.00000,-0.66325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
4,0.000000,0.000000,0.00000,0.00000,-23.310725,-0.000000,-0.000000,-0.000000,15.60000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,15.000000,0.000000,0.00000,0.00000,-0.000000,-23.310725,-0.000000,-0.000000,0.00000,15.60000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.00000,0.00000,-0.000000,-0.000000,-23.310725,-0.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,15.000000,0.00000,0.00000,-0.000000,-0.000000,-0.000000,-23.310725,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.00000,0.100000,0.000000,0.000000,0.000000,-15.66325,-0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,15.00000,0.00000,0.000000,0.100000,0.000000,0.000000,-0.00000,-15.66325,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [105]:
# Q~k
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)
pd.DataFrame(matrQw_k[1])

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,34.836066,0.0,0.000000,0.0,0.405738,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0,34.836066,0.0,0.000000,0.0,0.405738,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.081967,0.0,0.000000,0.0,1.040984,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0,0.081967,0.0,0.000000,0.0,1.040984,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,34.836066,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,34.836066,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.081967,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.081967,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
# Q^0
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQv_0)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Q_0
block0 = kron(np.eye(a), matrS1_0 * beta1)
block1 = kron(kron(np.eye(a), matrS2_0 * beta2), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0 * beta2), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQ_0)
print(kronsum(kronsum(kronsum(matrS1, matrS2), matrH0), matrD_0).shape)

(16, 16)


In [21]:
# Q_1
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])
pd.DataFrame(matrQ_1)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-28.310725,-0.000000,-0.000000,-0.000000,20.60000,0.00000,0.00000,0.00000,0.000000,0.000000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000000,-28.310725,-0.000000,-0.000000,0.00000,20.60000,0.00000,0.00000,0.000000,0.000000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.000000,-28.310725,-0.000000,0.00000,0.00000,20.60000,0.00000,0.000000,0.000000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.000000,-0.000000,-0.000000,-28.310725,0.00000,0.00000,0.00000,20.60000,0.000000,0.000000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.100000,0.000000,0.000000,0.000000,-20.66325,-0.00000,-0.00000,-0.00000,0.000000,0.000000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
5,0.000000,0.100000,0.000000,0.000000,-0.00000,-20.66325,-0.00000,-0.00000,0.000000,0.000000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
6,0.000000,0.000000,0.100000,0.000000,-0.00000,-0.00000,-20.66325,-0.00000,0.000000,0.000000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
7,0.000000,0.000000,0.000000,0.100000,-0.00000,-0.00000,-0.00000,-20.66325,0.000000,0.000000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,-38.310725,-0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,15.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,-0.000000,-38.310725,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+1):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

In [ ]:
# Поиск матрицы G
def algo1(matrG):
    res = matrQ_k[0]
    for k in range(1, n+1):
        res += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k))
    return res

initial_matrG = np.eye(matrQ_k[1].shape[0])
matrG = opt.newton_krylov(algo1, initial_matrG, iter=2000000, verbose=False)
pd.DataFrame(matrG)

In [34]:
temp_sum = matrQ_k[1]
for k in range(2, n+1):
    temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = matrG_0.dot(matrQv_0)
pd.DataFrame(matrG_0)

(56, 56)


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.023837,0.000000,-0.003037,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.023837,0.000000,-0.003037,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000320,0.000000,-0.909634,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000320,0.000000,-0.909634,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
matrQ_il = []
for i in range(0, n+1):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n+1):
            temp_matr = matrQw_k[l]
            for k in range(l+1, n+1):
                mult_matr = matrQw_k[k]
                for kk in range(k-1, l-1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
    else:
        for l in range(0, n+1):
            if l >= i:
                temp_matr = matrQ_k[l-i]
                for k in range(l+1, n+1):
                    mult_matr = matrQ_k[k-i]
                    for kk in range(k-1, l-1, -1):
                        if kk == 0:
                            mult_matr = mult_matr.dot(matrG_0)
                        else:
                            mult_matr = mult_matr.dot(matrG)
                    temp_matr += mult_matr
                matrQ_il[i].append(temp_matr)
            else:
                matrQ_il[i].append([])
    
print(len(matrQ_il[3]))
pd.DataFrame(matrQ_il[0][0])

4


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-8.310725,0.000000,0.60000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,-8.310725,0.00000,0.60000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.100000,0.000000,-0.66325,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
3,0.000000,0.100000,0.00000,-0.66325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
4,0.000000,0.000000,0.00000,0.00000,-23.310725,0.000000,0.000000,0.000000,15.60000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,15.000000,0.000000,0.00000,0.00000,0.000000,-23.310725,0.000000,0.000000,0.00000,15.60000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,-23.310725,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,15.000000,0.00000,0.00000,0.000000,0.000000,0.000000,-23.310725,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.00000,0.100000,0.000000,0.000000,0.000000,-15.66325,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,15.00000,0.00000,0.000000,0.100000,0.000000,0.000000,0.00000,-15.66325,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [140]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, n+1):
    temp_matr = (matrPhi_l[0].dot(matrQ_il[0][l])).dot(la.inv(-matrQ_il[l][l]))
    print(temp_matr.shape)
    for i in range(1, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += (matrPhi_l[i].dot(matrQ_il[i][l])).dot(la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)
pd.DataFrame(matrPhi_l[1])

(36, 56)
(36, 56)
(36, 56)


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-3.999981e-06,0.0,8.799936e-08,0.0,-1.375632e-04,0.0,-1.445981e-06,0.0,-5.966388e-22,0.000000e+00,...,1.141666e-09,4.282976e-23,-3.424993e-07,8.300508e-27,-6.828052e-06,1.648787e-21,-2.346078e-09,-1.383849e-26,-4.460177e-08,-2.830958e-23
1,-1.029449e-07,0.0,-6.289101e-06,0.0,-4.787026e-06,0.0,7.272842e-05,0.0,5.982933e-22,0.000000e+00,...,-3.551671e-09,-3.114167e-21,-8.251074e-09,-5.698184e-25,-1.677627e-07,-2.286109e-23,2.950170e-09,1.011117e-24,5.605438e-08,2.070391e-21
2,-1.000223e-08,0.0,2.344393e-10,0.0,-3.399045e-07,0.0,-3.648726e-09,0.0,-1.351746e-24,0.000000e+00,...,3.120514e-12,1.138884e-25,-1.023246e-09,2.191804e-29,-1.491260e-08,5.888563e-24,-7.002293e-12,-3.706145e-29,-1.327669e-10,-7.552762e-26
3,-2.560481e-10,0.0,-1.526667e-08,0.0,-1.132966e-08,0.0,1.483134e-07,0.0,1.455751e-24,0.000000e+00,...,-8.973777e-12,-7.564447e-24,-2.383272e-11,-1.384088e-27,-3.810243e-10,-2.800084e-26,7.075803e-12,2.456723e-27,1.340650e-10,5.031299e-24
4,2.600087e-24,0.0,7.459594e-25,0.0,-5.307317e-21,0.0,-9.789842e-25,0.0,-3.454068e-06,0.000000e+00,...,1.809494e-24,-2.531046e-24,-6.186625e-24,3.245539e-24,-3.022801e-22,8.473234e-23,-4.485603e-26,3.468402e-27,-6.050769e-24,5.417095e-24
5,1.768808e-05,0.0,9.027579e-08,0.0,8.995376e-03,0.0,-5.147540e-05,0.0,-2.737462e-05,-2.601475e-04,...,-3.148878e-09,-1.364860e-22,2.596215e-05,-1.296080e-22,4.795610e-04,-1.131818e-19,1.714308e-07,1.824759e-24,3.256070e-06,5.748974e-22
6,-8.130139e-24,0.0,-7.189418e-22,0.0,-2.978488e-21,0.0,-1.911853e-20,0.0,9.048571e-08,0.000000e+00,...,9.895062e-22,-2.572162e-22,-3.528757e-24,4.572422e-25,-1.954073e-23,-1.015771e-22,-5.493911e-25,5.276879e-25,-1.876566e-21,1.285102e-21
7,7.921158e-08,0.0,2.422638e-05,0.0,-1.751142e-04,0.0,1.318670e-02,0.0,-4.464718e-07,0.000000e+00,...,-1.543743e-08,-3.232234e-20,2.234493e-07,3.198977e-23,3.836095e-06,3.540236e-21,5.573149e-07,3.306795e-24,1.058369e-05,4.826881e-23
8,2.701593e-26,0.0,-2.852254e-26,0.0,-5.611995e-23,0.0,-5.557266e-26,0.0,-8.471111e-09,0.000000e+00,...,4.444382e-26,-1.569036e-26,-6.541351e-26,-3.085329e-26,-3.367234e-24,1.068010e-24,-4.074463e-28,1.397237e-29,-1.891972e-26,1.204275e-27
9,4.080843e-08,0.0,2.279089e-10,0.0,2.061454e-05,0.0,-1.188663e-07,0.0,-6.365536e-08,-6.034711e-07,...,-6.948529e-12,-3.288571e-25,5.951949e-08,-4.055905e-25,1.098691e-06,-2.679488e-22,3.929728e-10,4.317047e-27,7.464014e-09,1.337230e-24


In [185]:
matr_a = np.transpose(- matrQ_il[0][0])
matr_b = np.zeros((matr_a.shape[0], 1))
p0 = la.lstsq(matr_a, matr_b)[3]
p0 = np.transpose(p0)

print(type(a), type(R_))

vect_e1 = np.array([[1.] for i in range(0, p0.shape[0])])
print(vect_e1)
temp_vect = vect_e1
for l in range(1, n+1):
    vect_e2 = np.array([[1.] for i in range(0, matrPhi_l[l].shape[1])])
    print(matrPhi_l[l].shape)
    temp_vect += matrPhi_l[l].dot(vect_e2)
res = p0.dot(temp_vect)[0]
p0 /= res
print(p0.sum())

<class 'int'> <class 'int'>
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]
(36, 56)
(36, 56)
(36, 56)
0.999056484175


In [186]:
vect_p_l = [p0]
for l in range(1, n+1):
    vect_p_l.append(vect_p_l[0].dot(matrPhi_l[l]))
    print(vect_p_l[l])

[  5.80476105e-07   0.00000000e+00   2.42236375e-07   0.00000000e+00
   5.36596224e-04   0.00000000e+00   7.04616855e-04   0.00000000e+00
  -3.36507306e-06  -1.66148119e-05   7.12989426e-23  -4.25767418e-22
  -6.01466111e-06  -1.37696532e-05   7.79931540e-23  -3.52856931e-22
   5.87083075e-04  -3.76679897e-04   4.74555832e-20  -1.66885331e-20
  -1.04138125e-04  -3.12175115e-04   1.41246512e-20  -1.38306818e-20
  -5.31688974e-07   1.01916534e-06  -4.58373039e-23   5.48389530e-23
  -5.81924491e-07  -4.46179683e-07   3.17975330e-26  -7.84325582e-25
   2.76036743e-06   1.15699833e-05  -7.15660126e-21   1.76479595e-21
  -1.52173803e-07  -4.19685435e-07  -2.56703942e-24  -3.78859274e-24
  -2.11360607e-08  -2.12077107e-08  -2.13768319e-07  -1.78458237e-08
  -1.36349350e-08  -1.26171171e-08  -1.33816390e-08  -9.62271032e-09
   1.58882900e-06  -7.35704347e-09   2.94075651e-05  -6.18382534e-09
   3.56554789e-08  -4.37305065e-09   7.62435366e-07  -3.32935124e-09]
[ -1.04195151e-07   0.00000000e+0

In [188]:
# Условие эргодичности
block00 = kronsum(matrH0, matrS1) + kron(np.eye(V_), matrS1_0.dot(beta1))
block03 = kron(kron(kron(matrH1, M1_e), tau), alpha1)
block10 = kron(kron(np.eye(V_), M2_e.dot(beta1)), matrA2_0)
block11 = kronsum(kronsum(matrH0, matrS2), matrA2) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(L2))
block12 = kron(kron(kron(matrH1, np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(V_), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(matrH, matrS2), matrT) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(R))
block30 = kron(kron(kron(np.eye(V_), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(V_), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(matrH, matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block00.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))
matrGamma = np.bmat([[block00, block01, block02, block03],
                     [block10, block11, block12, block13],
                     [block20, block21, block22, block23],
                     [block30, block31, block32, block33]])
pd.DataFrame(matrGamma)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-28.110725,0.000000,20.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
1,20.000000,-28.110725,0.00000,20.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
2,0.000000,0.000000,-20.26325,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
3,0.000000,0.000000,20.00000,-20.26325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
4,0.000000,0.000000,0.00000,0.00000,-38.110725,0.000000,0.000000,0.000000,30.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,15.000000,0.000000,0.00000,0.00000,0.000000,-38.110725,0.000000,0.000000,0.00000,30.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.00000,0.00000,15.000000,0.000000,-38.110725,0.000000,0.00000,0.00000,...,0.002696,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,15.000000,0.000000,0.00000,0.00000,0.000000,15.000000,0.000000,-38.110725,0.00000,0.00000,...,0.002696,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,-30.26325,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,15.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,-30.26325,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [199]:
matr_a = np.transpose(matrGamma)
matr_b = np.zeros((matr_a.shape[0], 1))
x = la.lstsq(matr_a, matr_b)[3]
vect_e = np.array([[1.] for i in range(0, x.shape[0])])
res = x.dot(vect_e)[0]
x /= res
print('x = ', x)
x1 = x[0:V_*M1]
x2 = x[V_*M1:V_*M1 + V_*M2*L2]
x3 = x[V_*M1 + V_*M2*L2: V_*M1 + V_*M2*L2 + V_*M2*R]

print('x1 = ', x1)
print('x2 = ', x2)
print('x3 = ', x3)

x =  [ 0.10903372  0.0918408   0.08713946  0.08592118  0.07176864  0.06789743
  0.06784861  0.0664511   0.06274937  0.06141911  0.06061669  0.02453839
  0.0193178   0.01555679  0.01523008  0.01407867  0.01218539  0.01208218
  0.01130095  0.0095045   0.00876544  0.0075348   0.00644297  0.0044236
  0.00380481  0.00147295  0.00066339  0.00041115]
x1 =  [ 0.10903372  0.0918408   0.08713946  0.08592118]
x2 =  [ 0.07176864  0.06789743  0.06784861  0.0664511   0.06274937  0.06141911
  0.06061669  0.02453839]
x3 =  [ 0.0193178   0.01555679  0.01523008  0.01407867  0.01218539  0.01208218
  0.01130095  0.0095045 ]


In [201]:
e_V_ = np.array([[1.] for i in range(0, V_)])
e_R = np.array([[1.] for i in range(0, R)])
pi1 = x1.dot(kron(e_V_, np.eye(M1)))
pi2 = x2.dot(kron(kron(e_V_, np.eye(M2)), L2_e))
pi3 = x3.dot(kron(kron(e_V_, np.eye(M2)), e_R))
print('pi1 = ', pi1)
print('pi2 = ', pi2)
print('pi3 = ', pi3)

pi1 =  [ 0.19617318  0.17776197]
pi2 =  [ 0.26383455  0.21945479]
pi3 =  [ 0.05914218  0.0501142 ]


In [212]:
rho = pi1.dot(matrS1_0) + (pi2 + pi3).dot(matrS2_0)
rho = rho.tolist()[0][0]
print('rho = ', rho)

rho =  7.598774387205819
